In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
train_df = pd.read_csv('kc_house_data_train.csv', index_col = 0)
train_df.drop(train_df.loc[train_df['bedrooms'].eq(33)].index, axis = 0, inplace=True)
train_df['price_log'] = np.log(train_df['price'])

In [3]:
def get_ziprating(train_df):
    _grouped_qcut = pd.qcut(train_df.groupby(['zipcode'])['price'].mean(),
                            q=10, 
                            labels=['zip_lowest2','zip_lowest','zip_low2','zip_low','zip_medium2','zip_medium','zip_high2','zip_high','zip_highest2','zip_highest']).reset_index()
    #                                'zip_lowest3','zip_lowest4','zip_low3','zip_low4','zip_medium3','zip_medium4','zip_high3','zip_hig4','zip_highest3','zip_highest4']).reset_index()
    d = pd.Series(data= _grouped_qcut['price'].values,
                  index = _grouped_qcut['zipcode']).to_dict()
    return d

In [4]:
d = get_ziprating(train_df)

In [5]:
def cleaning_dataframe(train_df):    
    train_df['is_renovated'] = np.where(train_df['yr_renovated'].gt(0), 1, 0)
    train_df['zipcode_rating'] = train_df['zipcode'].map(d)
    train_df['is_viewed'] = np.where(train_df['view'] > 0, 1,0)
    train_df['sqft_living_log'] = np.log(train_df['sqft_living'])
    train_df['sqft_lot_log'] = np.log(train_df['sqft_lot'])
    train_df['sqft_above_log'] = np.log(train_df['sqft_above'])
    train_df['is_basement'] = np.where(train_df['sqft_basement'].gt(0), 1, 0)
    train_df['sqft_living15_log'] = np.log(train_df['sqft_living15'])
    train_df['sqft_lot15_log'] = np.log(train_df['sqft_lot15'])
    train_df['lat_log'] = np.log(train_df['lat'])
    train_df['long_log'] = np.log(train_df['long'] * -1)
    train_df['bath_cat'] = pd.cut(train_df['bathrooms'], 
                                  bins = [0, 1.51, 2.51, 3.76, 4.76,20], 
                                  labels = ['0_1_bath','1.5_bath','2_2.5_bath','3_more_bath','more'])
    train_df['yr_decade'] = pd.cut(x= train_df['yr_built'], 
                                   bins =[1899, 1909, 1919, 1929, 1939, 1949, 1959, 1969, 1979, 1989, 1999,2009,2019],
                                  labels= ['1900s', '1910s', '1920s', '1930s','1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s'])
    train_df['grade_cat'] = pd.cut(x= train_df['grade'],
                                  bins=[0,6,7,8,14],
                                  labels=['bad','bad-good','good','great'])
    train_df['bedrooms_cat'] = pd.cut(x=train_df['bedrooms'],
          bins=[-1,1.01,3.01,5.01,12],
          labels=['bed_0_1','bed_2_3','bed_4_5','bed_6_on'])
    train_df['sqft_living_diff'] = train_df['sqft_living'] - train_df['sqft_living15']
    train_df['sqft_lot_1/n_log'] = np.log(1/train_df['sqft_lot'])
    train_df['sqft_lot15_1/n_log'] = np.log(1/train_df['sqft_lot15'])
    train_df['sqft_living_1/n_log'] = np.log(1/train_df['sqft_living'])
    train_df['sqft_living15_1/n_log'] = np.log(1/train_df['sqft_living15'])
    train_df['building_age_log'] = np.log(2020 - train_df['yr_built'])
    train_df['building_age'] = (2020 - train_df['yr_built'])
    train_df['floors_cat'] = train_df['floors']
    train_df['condition_cat'] = train_df['condition']
    train_df['grade_cat'] = train_df['grade']
    train_df['lat_cos'] = np.cos(train_df['lat'])
    train_df['lon_cos'] = np.cos(train_df['long'])
    train_df['lat_sin'] = np.sin(train_df['lat'])
    train_df['lon_sin'] = np.sin(train_df['long'])
    train_df["rot45X"] = .707* train_df["long"] + .707* train_df["lat"] 
    train_df["rot45Y"] = .707* train_df["long"] - .707* train_df["lat"]
    train_df["rot30X"] = (1.732/2)* train_df["lat"] + (1./2)* train_df["long"] 
    train_df["rot30Y"] = (1.732/2)* train_df["long"] - (1./2)* train_df["lat"]
    train_df["rot60X"] = (1./2)* train_df["lat"] + (1.732/2)* train_df["long"] 
    train_df["rot60Y"] = (1./2)* train_df["long"] - (1.732/2)* train_df["lat"]

    train_df["radial_r"] = np.sqrt( np.power(train_df["long"],2) + np.power(train_df["lat"],2) )
    
    return train_df

In [6]:
train_df = cleaning_dataframe(train_df)

In [7]:
_train_df = pd.get_dummies(train_df, 
#                prefix=['bedrooms', 'floors','condition', 'grade', 'zipcode_rating','yr_decade','bath_cat'], 
#                columns =['bedrooms_cat','floors','condition','zipcode_rating','yr_decade','bath_cat'], drop_first=True
                          columns =['zipcode_rating','yr_decade','bedrooms_cat','bath_cat','floors_cat','condition_cat', 'grade_cat','zipcode'], drop_first=True)


_train_df.drop(columns=['id','date','price','sqft_basement','yr_built','yr_renovated','price_log',
                       'view'], inplace=True)

In [8]:
_train_df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_above,lat,long,sqft_living15,sqft_lot15,is_renovated,is_viewed,sqft_living_log,sqft_lot_log,sqft_above_log,is_basement,sqft_living15_log,sqft_lot15_log,lat_log,long_log,sqft_living_diff,sqft_lot_1/n_log,sqft_lot15_1/n_log,sqft_living_1/n_log,sqft_living15_1/n_log,building_age_log,building_age,lat_cos,lon_cos,lat_sin,lon_sin,rot45X,rot45Y,rot30X,rot30Y,rot60X,rot60Y,radial_r,zipcode_rating_zip_high2,zipcode_rating_zip_highest,zipcode_rating_zip_highest2,zipcode_rating_zip_low,zipcode_rating_zip_low2,zipcode_rating_zip_lowest,zipcode_rating_zip_lowest2,zipcode_rating_zip_medium,zipcode_rating_zip_medium2,yr_decade_1910s,yr_decade_1920s,yr_decade_1930s,yr_decade_1940s,yr_decade_1950s,yr_decade_1960s,yr_decade_1970s,yr_decade_1980s,yr_decade_1990s,yr_decade_2000s,yr_decade_2010s,bedrooms_cat_bed_2_3,bedrooms_cat_bed_4_5,bedrooms_cat_bed_6_on,bath_cat_1.5_bath,bath_cat_2_2.5_bath,bath_cat_3_more_bath,bath_cat_more,floors_cat_1.5,floors_cat_2.0,floors_cat_2.5,floors_cat_3.0,floors_cat_3.5,condition_cat_2,condition_cat_3,condition_cat_4,condition_cat_5,grade_cat_3,grade_cat_4,grade_cat_5,grade_cat_6,grade_cat_7,grade_cat_8,grade_cat_9,grade_cat_10,grade_cat_11,grade_cat_12,grade_cat_13,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98022,zipcode_98023,zipcode_98024,zipcode_98027,zipcode_98028,zipcode_98029,zipcode_98030,zipcode_98031,zipcode_98032,zipcode_98033,zipcode_98034,zipcode_98038,zipcode_98039,zipcode_98040,zipcode_98042,zipcode_98045,zipcode_98052,zipcode_98053,zipcode_98055,zipcode_98056,zipcode_98058,zipcode_98059,zipcode_98065,zipcode_98070,zipcode_98072,zipcode_98074,zipcode_98075,zipcode_98077,zipcode_98092,zipcode_98102,zipcode_98103,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,4,2.25,2070,8893,2.0,0,4,8,2070,47.4388,-122.162,2390,7700,0,0,7.635304,9.093020,7.635304,0,7.779049,8.948976,3.859440,4.805348,-320,-9.093020,-8.948976,-7.635304,-7.779049,3.526361,34,-0.950824,-0.935857,-0.309731,-0.352380,-52.829302,-119.907766,-19.998999,-129.511692,-82.072892,-102.163001,131.049586,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5,3.00,2900,6730,1.0,0,5,8,1830,47.6784,-122.285,2370,6283,0,0,7.972466,8.814330,7.512071,1,7.770645,8.745603,3.864478,4.806354,530,-8.814330,-8.745603,-7.972466,-7.770645,3.761200,43,-0.850158,-0.972020,-0.526527,-0.234898,-52.746866,-120.164124,-19.853006,-129.738010,-82.059610,-102.431994,131.251099,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4,2.50,3770,10893,2.0,0,3,11,3770,47.5646,-122.129,3710,9685,0,1,8.234830,9.295876,8.234830,0,8.218787,9.178334,3.862089,4.805078,60,-9.295876,-9.178334,-8.234830,-8.218787,3.135494,23,-0.904449,-0.923721,-0.426582,-0.383066,-52.717031,-119.973375,-19.873556,-129.546014,-81.981414,-102.255444,131.064426,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,3.50,4560,14608,2.0,0,3,12,4560,47.6995,-122.228,4050,14226,0,1,8.425078,9.589325,8.425078,0,8.306472,9.562827,3.864921,4.805888,510,-9.

In [9]:
_train_df.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'condition', 'grade', 'sqft_above', 'lat',
       ...
       'zipcode_98146', 'zipcode_98148', 'zipcode_98155', 'zipcode_98166',
       'zipcode_98168', 'zipcode_98177', 'zipcode_98178', 'zipcode_98188',
       'zipcode_98198', 'zipcode_98199'],
      dtype='object', length=157)

In [10]:
_train_df.corr()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_above,lat,long,sqft_living15,sqft_lot15,is_renovated,is_viewed,sqft_living_log,sqft_lot_log,sqft_above_log,is_basement,sqft_living15_log,sqft_lot15_log,lat_log,long_log,sqft_living_diff,sqft_lot_1/n_log,sqft_lot15_1/n_log,sqft_living_1/n_log,sqft_living15_1/n_log,building_age_log,building_age,lat_cos,lon_cos,lat_sin,lon_sin,rot45X,rot45Y,rot30X,rot30Y,rot60X,rot60Y,radial_r,zipcode_rating_zip_high2,zipcode_rating_zip_highest,zipcode_rating_zip_highest2,zipcode_rating_zip_low,zipcode_rating_zip_low2,zipcode_rating_zip_lowest,zipcode_rating_zip_lowest2,zipcode_rating_zip_medium,zipcode_rating_zip_medium2,yr_decade_1910s,yr_decade_1920s,yr_decade_1930s,yr_decade_1940s,yr_decade_1950s,yr_decade_1960s,yr_decade_1970s,yr_decade_1980s,yr_decade_1990s,yr_decade_2000s,yr_decade_2010s,bedrooms_cat_bed_2_3,bedrooms_cat_bed_4_5,bedrooms_cat_bed_6_on,bath_cat_1.5_bath,bath_cat_2_2.5_bath,bath_cat_3_more_bath,bath_cat_more,floors_cat_1.5,floors_cat_2.0,floors_cat_2.5,floors_cat_3.0,floors_cat_3.5,condition_cat_2,condition_cat_3,condition_cat_4,condition_cat_5,grade_cat_3,grade_cat_4,grade_cat_5,grade_cat_6,grade_cat_7,grade_cat_8,grade_cat_9,grade_cat_10,grade_cat_11,grade_cat_12,grade_cat_13,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98022,zipcode_98023,zipcode_98024,zipcode_98027,zipcode_98028,zipcode_98029,zipcode_98030,zipcode_98031,zipcode_98032,zipcode_98033,zipcode_98034,zipcode_98038,zipcode_98039,zipcode_98040,zipcode_98042,zipcode_98045,zipcode_98052,zipcode_98053,zipcode_98055,zipcode_98056,zipcode_98058,zipcode_98059,zipcode_98065,zipcode_98070,zipcode_98072,zipcode_98074,zipcode_98075,zipcode_98077,zipcode_98092,zipcode_98102,zipcode_98103,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
bedrooms,1.000000,0.524028,0.588531,0.030669,0.179891,-0.008671,0.021438,0.366397,0.488412,-0.003970,0.129577,0.401063,0.028031,0.014177,0.082172,0.633859,0.185113,0.526884,0.163973,0.414661,0.162900,-0.003939,-0.129447,0.442267,-0.185113,-0.162900,-0.633859,-0.414661,-0.141755,-0.159920,-0.007367,0.087056,0.003401,-0.136584,0.096186,0.089505,0.066133,0.108691,0.117576,0.065411,-0.117424,-0.040748,0.079873,0.106695,-0.070291,-0.010302,-0.010404,-0.020717,-0.059958,0.015682,-0.049101,-0.063558,-0.051660,-0.162933,-0.044139,0.087875,0.072161,-0.005612,0.052856,0.034446,0.075746,-0.763526,0.720350,0.409930,0.064654,0.318172,0.179557,0.123166,0.040934,0.223139,0.063054,-0.076872,-0.008182,-0.052938,0.008808,-0.006546,0.017528,-0.027963,-0.076784,-0.118135,-0.243206,-0.103967,0.072517,0.164255,0.138968,0.120528,0.062351,0.042564,-0.006826,0.000533,0.062802,0.046068,0.074476,0.036195,0.047024,-0.003338,0.022031,-0.030334,0.000863,-0.028713,0.006249,-0.002176,0.023529,0.013501,0.007072,0.015667,0.016190,-0.003452,0.034162,0.019327,0.012404,0.039205,0.083829,0.002533,-0.019116,0.048655,-0.023913,-0.013815,0.008926,0.025043,0.054879,0.017399,-0.049347,0.007033,0.039762,0.067114,0.030242,0.015398,-0.004356,-0.062094,0.018623,-0.040000,-0.049910,-0.020439,-0.011348,0.017654,-0.030346,-0.040516,-0.062642,-0.050443,-0.007715,-0.023624,-0.023029,-0.074666,-0.035567,-0.048957,-0.009088,-0.030637,-0.011387,-0.008001,0.002038,-0.049057,-0.005016,-0.004414,0.002074,-0.022390,-0.017841
bathrooms,0.524028,1.000000,0.755266,0.085465,0.497958,0.065846,-0.122931,0.664186,0.686849,0.023099,0.221292,0.568073,0.086492,0.047615,0.176626,0.761051,0.101546,0.694594,0.162549,0.569047,0.093460,0.023169,-0.221201,0.506966,-0.101546,-0.093460,-0.761051,-0.569

In [11]:
_train_df.shape

(17289, 157)

In [12]:
to_remove = ['fake','list']
_d = {'fake_dict':9,'each':4}
_test_remove = []
while list(_d.values())[0] > 1:
    _df = _train_df.corr().abs().stack().reset_index().sort_values(0, ascending=False)
    _df['pairs'] = list(zip(_df.level_0, _df.level_1))
    _df.set_index(['pairs'], inplace=True)
    _df.drop(columns =['level_1', 'level_0'], inplace=True)
    _df.columns = ['cc']
    _df.drop_duplicates(inplace=True)
    _to_remove = _df[(_df.cc>.75) & (_df.cc<1)]
    to_remove = _to_remove.reset_index()
    d = {}
    for i in to_remove['pairs']:
        for each in i:
            if each not in d:
                d[each] = 1
            else:
                d[each] += 1


    _d = {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}
    l1 = list(_d.keys())[0]
    print('removing :',l1)
    _test_remove.append(l1)
    _train_df.drop(columns=[l1],inplace=True)
    

removing : long
removing : rot60Y
removing : long_log
removing : rot30X
removing : sqft_living
removing : lon_sin
removing : sqft_above
removing : lat
removing : radial_r
removing : lat_log
removing : rot30Y
removing : rot60X
removing : bedrooms_cat_bed_2_3
removing : sqft_above_log
removing : sqft_lot_log
removing : sqft_lot_1/n_log
removing : condition_cat_3
removing : lat_cos


In [13]:
_test_remove

['long',
 'rot60Y',
 'long_log',
 'rot30X',
 'sqft_living',
 'lon_sin',
 'sqft_above',
 'lat',
 'radial_r',
 'lat_log',
 'rot30Y',
 'rot60X',
 'bedrooms_cat_bed_2_3',
 'sqft_above_log',
 'sqft_lot_log',
 'sqft_lot_1/n_log',
 'condition_cat_3',
 'lat_cos']

In [14]:
_d

{'lat_cos': 1,
 'lat_sin': 1,
 'sqft_living15_log': 1,
 'sqft_living15': 1,
 'building_age_log': 1,
 'building_age': 1,
 'bathrooms': 1,
 'sqft_living_1/n_log': 1}

In [15]:
to_remove

,pairs,cc
0,"(lat_cos, lat_sin)",0.980141
1,"(sqft_living15_log, sqft_living15)",0.976771
2,"(building_age_log, building_age)",0.933952
3,"(bathrooms, sqft_living_1/n_log)",0.761051


In [16]:
_train_df.shape

(17289, 139)

In [17]:
X = _train_df.copy()
y = train_df['price_log']

In [18]:
poly_2 = PolynomialFeatures(degree=2, include_bias=False)
poly2_data = poly_2.fit_transform(X)
poly2_columns = poly_2.get_feature_names(X.columns)
df_poly2 = pd.DataFrame(poly2_data, columns = poly2_columns)
df_poly2

,bedrooms,bathrooms,sqft_lot,floors,waterfront,condition,grade,sqft_living15,sqft_lot15,is_renovated,is_viewed,sqft_living_log,is_basement,sqft_living15_log,sqft_lot15_log,sqft_living_diff,sqft_lot15_1/n_log,sqft_living_1/n_log,sqft_living15_1/n_log,building_age_log,building_age,lon_cos,lat_sin,rot45X,rot45Y,zipcode_rating_zip_high2,zipcode_rating_zip_highest,zipcode_rating_zip_highest2,zipcode_rating_zip_low,zipcode_rating_zip_low2,zipcode_rating_zip_lowest,zipcode_rating_zip_lowest2,zipcode_rating_zip_medium,zipcode_rating_zip_medium2,yr_decade_1910s,yr_decade_1920s,yr_decade_1930s,yr_decade_1940s,yr_decade_1950s,yr_decade_1960s,yr_decade_1970s,yr_decade_1980s,yr_decade_1990s,yr_decade_2000s,yr_decade_2010s,bedrooms_cat_bed_4_5,bedrooms_cat_bed_6_on,bath_cat_1.5_bath,bath_cat_2_2.5_bath,bath_cat_3_more_bath,bath_cat_more,floors_cat_1.5,floors_cat_2.0,floors_cat_2.5,floors_cat_3.0,floors_cat_3.5,condition_cat_2,condition_cat_4,condition_cat_5,grade_cat_3,grade_cat_4,grade_cat_5,grade_cat_6,grade_cat_7,grade_cat_8,grade_cat_9,grade_cat_10,grade_cat_11,grade_cat_12,grade_cat_13,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98022,zipcode_98023,zipcode_98024,zipcode_98027,zipcode_98028,zipcode_98029,zipcode_98030,zipcode_98031,zipcode_98032,zipcode_98033,zipcode_98034,zipcode_98038,zipcode_98039,zipcode_98040,zipcode_98042,zipcode_98045,zipcode_98052,zipcode_98053,zipcode_98055,zipcode_98056,zipcode_98058,zipcode_98059,zipcode_98065,zipcode_98070,zipcode_98072,zipcode_98074,zipcode_98075,zipcode_98077,zipcode_98092,zipcode_98102,zipcode_98103,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199,bedrooms^2,bedrooms bathrooms,bedrooms sqft_lot,bedrooms floors,bedrooms waterfront,bedrooms condition,bedrooms grade,bedrooms sqft_living15,bedrooms sqft_lot15,bedrooms is_renovated,bedrooms is_viewed,bedrooms sqft_living_log,bedrooms is_basement,bedrooms sqft_living15_log,bedrooms sqft_lot15_log,bedrooms sqft_living_diff,bedrooms sqft_lot15_1/n_log,bedrooms sqft_living_1/n_log,bedrooms sqft_living15_1/n_log,bedrooms building_age_log,bedrooms building_age,bedrooms lon_cos,bedrooms lat_sin,bedrooms rot45X,bedrooms rot45Y,bedrooms zipcode_rating_zip_high2,bedrooms zipcode_rating_zip_highest,bedrooms zipcode_rating_zip_highest2,bedrooms zipcode_rating_zip_low,bedrooms zipcode_rating_zip_low2,bedrooms zipcode_rating_zip_lowest,bedrooms zipcode_rating_zip_lowest2,bedrooms zipcode_rating_zip_medium,bedrooms zipcode_rating_zip_medium2,bedrooms yr_decade_1910s,bedrooms yr_decade_1920s,bedrooms yr_decade_1930s,bedrooms yr_decade_1940s,bedrooms yr_decade_1950s,bedrooms yr_decade_1960s,bedrooms yr_decade_1970s,bedrooms yr_decade_1980s,bedrooms yr_decade_1990s,bedrooms yr_decade_2000s,bedrooms yr_decade_2010s,bedrooms bedrooms_cat_bed_4_5,bedrooms bedrooms_cat_bed_6_on,bedrooms bath_cat_1.5_bath,bedrooms bath_cat_2_2.5_bath,bedrooms bath_cat_3_more_bath,bedrooms bath_cat_more,bedrooms floors_cat_1.5,bedrooms floors_cat_2.0,bedrooms floors_cat_2.5,bedrooms floors_cat_3.0,bedrooms floors_cat_3.5,bedrooms condition_cat_2,bedrooms condition_cat_4,bedrooms condition_cat_5,bedrooms grade_cat_3,bedrooms grade_cat_4,bedrooms grade_cat_5,bedrooms grade_cat_6,bedrooms grade_cat_7,bedrooms grade_cat_8,bedrooms grade_cat_9,bedrooms grade_cat_10,bedrooms grade_cat_11,bedrooms grade_cat_12,bedrooms grade_cat_13,bedrooms zipcode_98002,bedrooms zipcode_98003,bedrooms zipcode_98004,bedrooms zipcode_98005,bedrooms zipcode_98006,bedrooms zipcode_98007,bedrooms zipcode_98008,bedrooms zipcode_98010,bedrooms zipcode_98011,bedrooms zipcode_

In [19]:
_X_ = df_poly2.copy()

In [20]:
X_train, X_test, y_train, y_test = train_test_split(_X_, y, random_state=42, test_size=0.2)

In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(data=X_train,columns = _X_.columns)
X_test = scaler.transform(X_test)
X_test = pd.DataFrame(data=X_test,columns = _X_.columns)

In [22]:
selector = SelectKBest(f_regression, k=200)
selector.fit(X_train, y_train)

C:\Users\albert\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
C:\Users\albert\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\albert\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\albert\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


SelectKBest(k=200, score_func=<function f_regression at 0x0000000011A04438>)

In [23]:
selected_columns = X_train.columns[selector.get_support()]

In [24]:
lm_kbest = LinearRegression()
lm_kbest = lm_kbest.fit(X_train[selected_columns], y_train)
y_train_kbest = lm_kbest.predict(X_train[selected_columns])
np.sqrt(metrics.mean_squared_error(np.exp(y_train), np.exp(y_train_kbest)))

143257.15484128808

In [25]:
y_kbest = lm_kbest.predict(X_test[selected_columns])
np.sqrt(metrics.mean_squared_error(np.exp(y_test), np.exp(y_kbest)))

147167.18317489722

# LassoCV

In [26]:
from sklearn.linear_model import LassoCV

lassoCV_model = LassoCV(cv=5, random_state=42, verbose=1)
lassoCV_model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
...........................................................................................................................................................................................................................................................................................................C:\Users\albert\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.3462774493514189, tolerance: 0.3058141567364248
  tol, rng, random, positive)
.........................................................................................................................................................................................................[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 28.9min finished
C:\Users\albert\Anaconda3\lib\site-packages\sklearn\linear_mod

LassoCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=42, selection='cyclic',
        tol=0.0001, verbose=1)

In [27]:
y_train_lassocv_pred = lassoCV_model.predict(X_train)
y_test_lassocv_pred = lassoCV_model.predict(X_test)

# np.sqrt(metrics.mean_squared_error(y_train,y_train_pred_lassocv))

In [28]:
np.sqrt(metrics.mean_squared_error(np.exp(y_train),np.exp(y_train_lassocv_pred)))

103579.49087486637

In [29]:
np.sqrt(metrics.mean_squared_error(np.exp(y_test),np.exp(y_test_lassocv_pred)))

133478.8387842201

In [30]:
r2 = lassoCV_model.score(X_train, y_train)

In [31]:
r2

0.9096716966675997

In [32]:
import pickle
with open('regression_lassoCVmodel2.pickle', 'wb') as f:
    pickle.dump(lassoCV_model, f)

In [33]:
with open('regression_lassoCVmodel2.pickle', 'rb') as file:
    reg2 = pickle.load(file)

In [34]:
test_df = pd.read_csv('kc_house_data_test_features.csv',index_col=0)
d = get_ziprating(train_df)
test_df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [35]:
test_df = cleaning_dataframe(test_df)

In [36]:
test_df

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,is_renovated,zipcode_rating,is_viewed,sqft_living_log,sqft_lot_log,sqft_above_log,is_basement,sqft_living15_log,sqft_lot15_log,lat_log,long_log,bath_cat,yr_decade,grade_cat,bedrooms_cat,sqft_living_diff,sqft_lot_1/n_log,sqft_lot15_1/n_log,sqft_living_1/n_log,sqft_living15_1/n_log,building_age_log,building_age,floors_cat,condition_cat,lat_cos,lon_cos,lat_sin,lon_sin,rot45X,rot45Y,rot30X,rot30Y,rot60X,rot60Y,radial_r
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,0,zip_medium,0,7.727535,9.350102,7.339538,1,7.610853,9.298168,3.865118,4.805995,1.5_bath,1960s,8,bed_4_5,250,-9.350102,-9.298168,-7.727535,-7.610853,3.970292,53,1.0,3,-0.833706,-0.960747,-0.552208,-0.277426,-52.694195,-120.154579,-19.804593,-129.715156,-82.006256,-102.436407,131.221192
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,0,zip_medium,0,7.727535,9.350102,7.339538,1,7.610853,9.298168,3.865118,4.805995,1.5_bath,1960s,8,bed_4_5,250,-9.350102,-9.298168,-7.727535,-7.610853,3.970292,53,1.0,3,-0.833706,-0.960747,-0.552208,-0.277426,-52.694195,-120.154579,-19.804593,-129.715156,-82.006256,-102.436407,131.221192
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576,0,zip_high2,0,7.293018,7.483807,7.056175,1,7.293018,7.362645,3.861723,4.804005,1.5_bath,2000s,8,bed_2_3,0,-7.483807,-7.362645,-7.293018,-7.293018,2.708050,15,2.0,3,-0.911734,-0.865768,-0.410781,-0.500446,-52.636716,-119.868456,-19.823125,-129.423868,-81.876668,-102.174875,130.936046
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565,0,zip_high,0,7.154615,9.692767,6.937314,1,7.056175,9.265302,3.865826,4.804603,1.5_bath,1970s,8,bed_2_3,120,-9.692767,-9.265302,-7.154615,-7.056175,3.784190,44,1.0,3,-0.814569,-0.899962,-0.580066,-0.435968,-52.550108,-120.058286,-19.690322,-129.584836,-81.842136,-102.380678,131.075148
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916,0,zip_medium2,0,7.948032,9.002824,7.948032,0,7.948032,8.976641,3.860441,4.805168,2_2.5_bath,2000s,8,bed_4_5,0,-9.002824,-8.976641,-7.948032,-7.948032,2.708050,15,2.0,3,-0.935045,-0.927879,-0.354529,-0.372882,-52.780166,-119.925794,-19.946864,-129.516390,-82.030090,-102.193136,131.046283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,263000018,20140521T000000,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509,0,zip_high2,0,7.333023,7.030857,7.333023,0,7.333023,7.319202,3.864917,4.806853,1.5_bath,2000s,8,bed_2_3,0,-7.030857,-7.319202,-7.333023,-7.333023,2.397895,11,3.0,3,-0.838969,-0.984532,-0.544179,-0.175205,-52.775217,-120.222027,-19.865406,-129.801286,-82.101986,-102.480594,131.315524
4319,6600060120,20150223T000000,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200,0,zip_low2,0,7.745003,8.667852,7.745003,0,7.512071,8.881836,3.860955,4.806984,1.5_bath,2010s,8,bed_4_5,480,-8.667852,-8.881836,-7.745003,-7.512071,1.791759,6,2.0,3,-0.926117,-0.987209,-0.377236,-0.159430,-52.919869,-120.099999,-20.036734,-129.720842,-82.210142,-102.325266,131.262050
4320,1523300141,20140623T000000,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007,0,zip_high,0,6.927558,7.207860,6.927558,0,6.927558,7.604396,3.862715,4.806469,0_1_bath,2000s,7,bed_2_3,0,-7.207860,-7.604396,-6.927558,-6.927558,2.397895,11,2.0,3,-0.891337,-0.975213,-0.453341,-0.221267,-52.816152,-120.114634,-19.932750,-129.708134,-82.113734,-102.366250,131.233655
4321,291310100,20150116T000000,3,2.5

In [37]:
_test_df = pd.get_dummies(test_df, 
#                prefix=['bedrooms', 'floors','condition', 'grade', 'zipcode_rating','yr_decade','bath_cat'], 
#                columns =['bedrooms_cat','floors','condition','zipcode_rating','yr_decade','bath_cat'], drop_first=True
                          columns =['zipcode_rating','yr_decade','bedrooms_cat','bath_cat','floors_cat','condition_cat', 'grade_cat','zipcode'], drop_first=True)


_test_df.drop(columns=['id','date','sqft_basement','yr_built','yr_renovated',
                       'view'], inplace=True)

In [38]:
_test_df.drop(columns=_test_remove)

,bedrooms,bathrooms,sqft_lot,floors,waterfront,condition,grade,sqft_living15,sqft_lot15,is_renovated,is_viewed,sqft_living_log,is_basement,sqft_living15_log,sqft_lot15_log,sqft_living_diff,sqft_lot15_1/n_log,sqft_living_1/n_log,sqft_living15_1/n_log,building_age_log,building_age,lon_cos,lat_sin,rot45X,rot45Y,zipcode_rating_zip_high2,zipcode_rating_zip_highest,zipcode_rating_zip_highest2,zipcode_rating_zip_low,zipcode_rating_zip_low2,zipcode_rating_zip_lowest,zipcode_rating_zip_lowest2,zipcode_rating_zip_medium,zipcode_rating_zip_medium2,yr_decade_1910s,yr_decade_1920s,yr_decade_1930s,yr_decade_1940s,yr_decade_1950s,yr_decade_1960s,yr_decade_1970s,yr_decade_1980s,yr_decade_1990s,yr_decade_2000s,yr_decade_2010s,bedrooms_cat_bed_4_5,bedrooms_cat_bed_6_on,bath_cat_1.5_bath,bath_cat_2_2.5_bath,bath_cat_3_more_bath,bath_cat_more,floors_cat_1.5,floors_cat_2.0,floors_cat_2.5,floors_cat_3.0,floors_cat_3.5,condition_cat_2,condition_cat_4,condition_cat_5,grade_cat_4,grade_cat_5,grade_cat_6,grade_cat_7,grade_cat_8,grade_cat_9,grade_cat_10,grade_cat_11,grade_cat_12,grade_cat_13,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98022,zipcode_98023,zipcode_98024,zipcode_98027,zipcode_98028,zipcode_98029,zipcode_98030,zipcode_98031,zipcode_98032,zipcode_98033,zipcode_98034,zipcode_98038,zipcode_98039,zipcode_98040,zipcode_98042,zipcode_98045,zipcode_98052,zipcode_98053,zipcode_98055,zipcode_98056,zipcode_98058,zipcode_98059,zipcode_98065,zipcode_98070,zipcode_98072,zipcode_98074,zipcode_98075,zipcode_98077,zipcode_98092,zipcode_98102,zipcode_98103,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,4,2.50,11500,1.0,0,3,8,2020,10918,0,0,7.727535,1,7.610853,9.298168,250,-9.298168,-7.727535,-7.610853,3.970292,53,-0.960747,-0.552208,-52.694195,-120.154579,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4,2.50,11500,1.0,0,3,8,2020,10918,0,0,7.727535,1,7.610853,9.298168,250,-9.298168,-7.727535,-7.610853,3.970292,53,-0.960747,-0.552208,-52.694195,-120.154579,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,2.50,1779,2.0,0,3,8,1470,1576,0,0,7.293018,1,7.293018,7.362645,0,-7.362645,-7.293018,-7.293018,2.708050,15,-0.865768,-0.410781,-52.636716,-119.868456,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,1.75,16200,1.0,0,3,8,1160,10565,0,0,7.154615,1,7.056175,9.265302,120,-9.265302,-7.154615,-7.056175,3.784190,44,-0.899962,-0.580066,-52.550108,-120.058286,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,2.75,8126,2.0,0,3,8,2830,7916,0,0,7.948032,0,7.948032,8.976641,0,-8.976641,-7.948032,-7.948032,2.708050,15,-0.927879,-0.354529,-52.780166,-119.925794,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,

In [39]:
_train_df

,bedrooms,bathrooms,sqft_lot,floors,waterfront,condition,grade,sqft_living15,sqft_lot15,is_renovated,is_viewed,sqft_living_log,is_basement,sqft_living15_log,sqft_lot15_log,sqft_living_diff,sqft_lot15_1/n_log,sqft_living_1/n_log,sqft_living15_1/n_log,building_age_log,building_age,lon_cos,lat_sin,rot45X,rot45Y,zipcode_rating_zip_high2,zipcode_rating_zip_highest,zipcode_rating_zip_highest2,zipcode_rating_zip_low,zipcode_rating_zip_low2,zipcode_rating_zip_lowest,zipcode_rating_zip_lowest2,zipcode_rating_zip_medium,zipcode_rating_zip_medium2,yr_decade_1910s,yr_decade_1920s,yr_decade_1930s,yr_decade_1940s,yr_decade_1950s,yr_decade_1960s,yr_decade_1970s,yr_decade_1980s,yr_decade_1990s,yr_decade_2000s,yr_decade_2010s,bedrooms_cat_bed_4_5,bedrooms_cat_bed_6_on,bath_cat_1.5_bath,bath_cat_2_2.5_bath,bath_cat_3_more_bath,bath_cat_more,floors_cat_1.5,floors_cat_2.0,floors_cat_2.5,floors_cat_3.0,floors_cat_3.5,condition_cat_2,condition_cat_4,condition_cat_5,grade_cat_3,grade_cat_4,grade_cat_5,grade_cat_6,grade_cat_7,grade_cat_8,grade_cat_9,grade_cat_10,grade_cat_11,grade_cat_12,grade_cat_13,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98022,zipcode_98023,zipcode_98024,zipcode_98027,zipcode_98028,zipcode_98029,zipcode_98030,zipcode_98031,zipcode_98032,zipcode_98033,zipcode_98034,zipcode_98038,zipcode_98039,zipcode_98040,zipcode_98042,zipcode_98045,zipcode_98052,zipcode_98053,zipcode_98055,zipcode_98056,zipcode_98058,zipcode_98059,zipcode_98065,zipcode_98070,zipcode_98072,zipcode_98074,zipcode_98075,zipcode_98077,zipcode_98092,zipcode_98102,zipcode_98103,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,4,2.25,8893,2.0,0,4,8,2390,7700,0,0,7.635304,0,7.779049,8.948976,-320,-8.948976,-7.635304,-7.779049,3.526361,34,-0.935857,-0.309731,-52.829302,-119.907766,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5,3.00,6730,1.0,0,5,8,2370,6283,0,0,7.972466,1,7.770645,8.745603,530,-8.745603,-7.972466,-7.770645,3.761200,43,-0.972020,-0.526527,-52.746866,-120.164124,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4,2.50,10893,2.0,0,3,11,3710,9685,0,1,8.234830,0,8.218787,9.178334,60,-9.178334,-8.234830,-8.218787,3.135494,23,-0.923721,-0.426582,-52.717031,-119.973375,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,3.50,14608,2.0,0,3,12,4050,14226,0,1,8.425078,0,8.306472,9.562827,510,-9.562827,-8.425078,-8.306472,3.401197,30,-0.957060,-0.544347,-52.691649,-120.138742,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3,2.50,5376,2.0,0,3,9,2250,4050,0,0,7.843849,0,7.718685,8.306472,300,-8.306472,-7.843849,-7.718685,2.772589,16,-0.905129,-0.514831,-52.613738,-120.011624,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...

In [40]:
test_df['grade_cat'].unique()

array([ 8,  7,  6,  9, 10,  5,  4, 11, 12, 13,  1], dtype=int64)

In [41]:
train_df['grade_cat'].unique()

array([ 8, 11, 12,  9,  7,  6, 10,  5, 13,  4,  3,  1], dtype=int64)

In [42]:
_test_df.columns.get_loc("grade_cat_4")

77

In [43]:
_test_df.insert(77, "grade_cat_3", 0)

In [44]:
_test_df.drop(columns=_test_remove, inplace=True)

In [45]:
_test_df_poly_data = poly_2.transform(_test_df)
_test_df_poly_columns = poly_2.get_feature_names(_test_df.columns)
_test_df_poly2 = pd.DataFrame(data=_test_df_poly_data,
                             columns=_test_df_poly_columns)
_test_df_poly2

,bedrooms,bathrooms,sqft_lot,floors,waterfront,condition,grade,sqft_living15,sqft_lot15,is_renovated,is_viewed,sqft_living_log,is_basement,sqft_living15_log,sqft_lot15_log,sqft_living_diff,sqft_lot15_1/n_log,sqft_living_1/n_log,sqft_living15_1/n_log,building_age_log,building_age,lon_cos,lat_sin,rot45X,rot45Y,zipcode_rating_zip_high2,zipcode_rating_zip_highest,zipcode_rating_zip_highest2,zipcode_rating_zip_low,zipcode_rating_zip_low2,zipcode_rating_zip_lowest,zipcode_rating_zip_lowest2,zipcode_rating_zip_medium,zipcode_rating_zip_medium2,yr_decade_1910s,yr_decade_1920s,yr_decade_1930s,yr_decade_1940s,yr_decade_1950s,yr_decade_1960s,yr_decade_1970s,yr_decade_1980s,yr_decade_1990s,yr_decade_2000s,yr_decade_2010s,bedrooms_cat_bed_4_5,bedrooms_cat_bed_6_on,bath_cat_1.5_bath,bath_cat_2_2.5_bath,bath_cat_3_more_bath,bath_cat_more,floors_cat_1.5,floors_cat_2.0,floors_cat_2.5,floors_cat_3.0,floors_cat_3.5,condition_cat_2,condition_cat_4,condition_cat_5,grade_cat_3,grade_cat_4,grade_cat_5,grade_cat_6,grade_cat_7,grade_cat_8,grade_cat_9,grade_cat_10,grade_cat_11,grade_cat_12,grade_cat_13,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98022,zipcode_98023,zipcode_98024,zipcode_98027,zipcode_98028,zipcode_98029,zipcode_98030,zipcode_98031,zipcode_98032,zipcode_98033,zipcode_98034,zipcode_98038,zipcode_98039,zipcode_98040,zipcode_98042,zipcode_98045,zipcode_98052,zipcode_98053,zipcode_98055,zipcode_98056,zipcode_98058,zipcode_98059,zipcode_98065,zipcode_98070,zipcode_98072,zipcode_98074,zipcode_98075,zipcode_98077,zipcode_98092,zipcode_98102,zipcode_98103,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199,bedrooms^2,bedrooms bathrooms,bedrooms sqft_lot,bedrooms floors,bedrooms waterfront,bedrooms condition,bedrooms grade,bedrooms sqft_living15,bedrooms sqft_lot15,bedrooms is_renovated,bedrooms is_viewed,bedrooms sqft_living_log,bedrooms is_basement,bedrooms sqft_living15_log,bedrooms sqft_lot15_log,bedrooms sqft_living_diff,bedrooms sqft_lot15_1/n_log,bedrooms sqft_living_1/n_log,bedrooms sqft_living15_1/n_log,bedrooms building_age_log,bedrooms building_age,bedrooms lon_cos,bedrooms lat_sin,bedrooms rot45X,bedrooms rot45Y,bedrooms zipcode_rating_zip_high2,bedrooms zipcode_rating_zip_highest,bedrooms zipcode_rating_zip_highest2,bedrooms zipcode_rating_zip_low,bedrooms zipcode_rating_zip_low2,bedrooms zipcode_rating_zip_lowest,bedrooms zipcode_rating_zip_lowest2,bedrooms zipcode_rating_zip_medium,bedrooms zipcode_rating_zip_medium2,bedrooms yr_decade_1910s,bedrooms yr_decade_1920s,bedrooms yr_decade_1930s,bedrooms yr_decade_1940s,bedrooms yr_decade_1950s,bedrooms yr_decade_1960s,bedrooms yr_decade_1970s,bedrooms yr_decade_1980s,bedrooms yr_decade_1990s,bedrooms yr_decade_2000s,bedrooms yr_decade_2010s,bedrooms bedrooms_cat_bed_4_5,bedrooms bedrooms_cat_bed_6_on,bedrooms bath_cat_1.5_bath,bedrooms bath_cat_2_2.5_bath,bedrooms bath_cat_3_more_bath,bedrooms bath_cat_more,bedrooms floors_cat_1.5,bedrooms floors_cat_2.0,bedrooms floors_cat_2.5,bedrooms floors_cat_3.0,bedrooms floors_cat_3.5,bedrooms condition_cat_2,bedrooms condition_cat_4,bedrooms condition_cat_5,bedrooms grade_cat_3,bedrooms grade_cat_4,bedrooms grade_cat_5,bedrooms grade_cat_6,bedrooms grade_cat_7,bedrooms grade_cat_8,bedrooms grade_cat_9,bedrooms grade_cat_10,bedrooms grade_cat_11,bedrooms grade_cat_12,bedrooms grade_cat_13,bedrooms zipcode_98002,bedrooms zipcode_98003,bedrooms zipcode_98004,bedrooms zipcode_98005,bedrooms zipcode_98006,bedrooms zipcode_98007,bedrooms zipcode_98008,bedrooms zipcode_98010,bedrooms zipcode_98011,bedrooms zipcode_

In [46]:
_test_df_poly2_scaled = scaler.transform(_test_df_poly2)

In [47]:
submission = np.exp(reg2.predict(_test_df_poly2_scaled))


In [48]:
submission_df = pd.DataFrame(data=submission,
            columns=['price'])

In [49]:
submission_df.to_csv('binya-albert-submission2.csv',index=False,header=True)